In [1]:
import os
from pathlib import Path
os.chdir(Path.cwd().parent)

In [ ]:
import pandas as pd
import numpy as np
from data_loader import scores_df
from fit import fit_statistical_model

anchor_benchmark = "Winogrande"
anchor_difficulty = 0
anchor_slope = 1
df1, df_cm1, df_db1 = fit_statistical_model(
    scores_df, 
    anchor_mode="benchmark", 
    anchor_benchmark=anchor_benchmark, 
    anchor_difficulty=anchor_difficulty, 
    anchor_slope=anchor_slope)

# Convert date strings to datetime objects
df_cm1['date_obj'] = pd.to_datetime(df_cm1['date'])

gpt_3_001_score = df_cm1[df_cm1['model'].str.contains('text-davinci-001')]['estimated_capability'].values[0]
gpt_3_002_score = df_cm1[df_cm1['model'].str.contains('text-davinci-002')]['estimated_capability'].values[0]
gpt_4_score = df_cm1[df_cm1['model'].str.contains('gpt-4-0613')]['estimated_capability'].values[0]
gpt_5_score = df_cm1[df_cm1['model'].str.contains('gpt-5-2025-08-07_medium')]['estimated_capability'].values[0]
o3_score = df_cm1[df_cm1['model'].str.contains('o3-2025-04-16_high')]['estimated_capability'].values[0]
print(f"GPT-3 (001): {gpt_3_001_score:.3}")
print(f"GPT-3 (002): {gpt_3_002_score:.3}")
print(f"GPT-4: {gpt_4_score:.3}")
print(f"o3: {o3_score:.3}")
print(f"GPT-5: {gpt_5_score:.3}")

null performances after coercion: 295
after saturation filter 1911
after filter num benchmarks 1473
after merge with model versions 1468
after merge with benchmark dates 1468
Original number of rows: 1468
Number of rows after aggregation: 1244
`ftol` termination condition is satisfied.
Function evaluations 38, initial cost 4.3493e+01, final cost 2.6360e+00, first-order optimality 1.17e-03.
GPT-3 (001): 0.954
GPT-3 (002): 1.49
GPT-4: 1.52
o3: 2.43
GPT-5: 2.65


In [ ]:
gpt_4_score

a = 54.2
b = 48
df_cm1['eci'] = a + b * df_cm1['estimated_capability']

print(f"GPT-3: {df_cm1[df_cm1['model'].str.contains('text-davinci-001')]['eci'].values[0]}")
print(f"GPT-4: {df_cm1[df_cm1['model'].str.contains('gpt-4-0613')]['eci'].values[0]}")


GPT-3: 100.00201500661473
GPT-4: 127.00073712587486


In [9]:
df_cm1[df_cm1['model'].str.contains('gpt-4')]

,model_id,estimated_capability,model,is_anchor,Model,date,date_obj,eci
218,m297,2.146531,gpt-4-0314,False,GPT-4,2023-03-14,2023-03-14,157.233468
346,m75,2.093878,gpt-4.5-preview-2025-02-27,False,GPT-4.5,2025-02-27,2025-02-27,154.706123
365,m92,2.089107,gpt-4.1-2025-04-14,False,GPT-4.1,2025-04-14,2025-04-14,154.477136
47,m141,2.086816,chatgpt-4o-03-27-2025,False,GPT-4o,2025-03-27,2025-03-27,154.367154
367,m94,2.032842,gpt-4.1-mini-2025-04-14,False,GPT-4.1 mini,2025-04-14,2025-04-14,151.776435
50,m144,1.895192,chatgpt-4o-01-29-2025,False,GPT-4o,2025-01-29,2025-01-29,145.169219
366,m93,1.863125,gpt-4.1-nano-2025-04-14,False,GPT-4.1 nano,2025-04-14,2025-04-14,143.630006
334,m64,1.794793,gpt-4o-2024-11-20,False,GPT-4o,2024-11-20,2024-11-20,140.350076
329,m6,1.784163,gpt-4o-2024-08-06,False,GPT-4o,2024-08-06,2024-08-06,139.839831
56,m15,1.767970,gpt-4o-2024-05-13,False,GPT-4o,2024-05-13,2024-05-13,139.062571


In [ ]:
df_cm1.to_csv('output/eci_scores.csv')